In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from ripple_heterogeneity.utils import (
    functions,
    loading,
    batch_analysis,
    add_new_deep_sup,
)
from ripple_heterogeneity.replay import replay_run, influence_of_layer_bias_on_replay
from ripple_heterogeneity.readout import readout_explained_var
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nelpy as nel
import nelpy.plotting as npl
from matplotlib.ticker import AutoMinorLocator
from sklearn.cluster import KMeans
import os
from scipy import stats

functions.set_plotting_defaults()
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
fig_save_path = r'Z:\home\ryanh\projects\ripple_heterogeneity\figures\panels'

c:\Users\Cornell\anaconda3\envs\ripple_diversity\lib\site-packages\nelpy\plotting\decoding.py:9: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.


In [3]:
save_path = (
    r"Z:\home\ryanh\projects\ripple_heterogeneity\readout_explained_var_ripples_extend_50ms_w_pairwise_corr"
)
ev_df, pairwise_corr_df = readout_explained_var.load_results(save_path)


In [4]:
ripple_df = pd.DataFrame()
for basepath in ev_df.basepath.unique():
    ripple_df = pd.concat([ripple_df,loading.load_ripples_events(basepath)],ignore_index=True)

In [ ]:
save_path = (
    r"Z:\home\ryanh\projects\ripple_heterogeneity\ripple_layer_bias"
)

batch_analysis.run(
    ev_df,
    save_path,
    influence_of_layer_bias_on_replay.run,
    parallel=True,
    overwrite=False,
    replay_df=ripple_df,
    n_shuffles=1000,
    q_perc=95
)
# load results
df_layer_bias = influence_of_layer_bias_on_replay.load_results(save_path)

In [ ]:
print(np.sum(df_layer_bias.pval_deep < 0.05),np.sum(df_layer_bias.pval_sup < 0.05),np.sum(df_layer_bias.pval_middle < 0.05))
np.sum(df_layer_bias.pval_deep < 0.05)/df_layer_bias.shape[0],np.sum(df_layer_bias.pval_sup < 0.05)/df_layer_bias.shape[0],np.sum(df_layer_bias.pval_middle < 0.05)/df_layer_bias.shape[0]


In [ ]:
sns.boxplot(x="sig_unit_bias",y="isns",data=df_layer_bias,showfliers=False)
# sns.stripplot(x="sig_unit_bias",y="effective",data=df_layer_bias)
plt.axhline(0,color='k',linestyle='--')
# plt.yscale("log")

In [ ]:
df_layer_bias.keys()


In [ ]:
plt.hist(df_layer_bias[df_layer_bias.pval_deep < 0.05].duration,100,density=True,alpha=0.5)
plt.hist(df_layer_bias[df_layer_bias.pval_sup < 0.05].duration,100,density=True,alpha=0.5)
# plt.hist(df_layer_bias[df_layer_bias.pval_middle < 0.05].duration,100,density=True,alpha=0.5)

plt.show()

In [ ]:
plt.hist(df_layer_bias.pval_deep,100,density=True)
plt.hist(df_layer_bias.pval_sup,100,density=True,alpha=0.5)
plt.hist(df_layer_bias.pval_middle,100,density=True,alpha=0.5)
# plt.xlim(0,.2)
plt.yscale("log")
df_layer_bias.keys()